In [2]:
from geopy.geocoders.googlev3 import GoogleV3
import shapefile
import pandas as pd
import numpy as np
import shapely
from shapely.geometry import Polygon
from shapely.geometry import Point
import requests
from pprint import pprint

<h3>The WalkScore API takes three required parameters: lat, long, address. 
<br>Strategy: pick a 'representative point' in each neighborhood polygon, get that point's lat/long, and then use the Google Maps API (through a geocoder) to determine the address of that point (or the first address available around that point, since there are likely going to be many.</3>

In [10]:
# Open up shapefiles
sf = shapefile.Reader('../downloads/sf_neighborhoods/geo_export_197f44fb-6cc0-472b-81f7-347deefb57df')
sea = shapefile.Reader('../downloads/seattle_neighborhoods/Neighborhoods')

# Desired output: ['Lower Haight', [(-122.4376483, 37.77310583), (-122.4376483, 37.77310583)]]

# Dictionary from city name (e.g. Seattle) to list of "Desired output" records,
# described above.
hoods_by_city = {}

def record_to_neighborhood(city, record):
    if city == 'Seattle':
        if record[5] != 'OOO' and record[5][1] != ' ':
            return record[5]
        else:
            return None
    elif city == 'San Francisco':
        return record[2]
    else:
        raise Exception("unsupported city: " + city)

# Make Polygon objects for SF and Seattle neighborhoods

for city, polyfile in (('San Francisco', sf), ('Seattle', sea)):
    for record, shape in zip(polyfile.records(), polyfile.shapes()):
        name = record_to_neighborhood(city, record)
        if name:
            if city not in hoods_by_city:
                hoods_by_city[city] = []
            hoods_by_city[city].append([name, Polygon([list(reversed(p)) for p in shape.points])])

<h3>In lieu of using a poorly designed walkscore Python client, let's use requests instead.</h3>

In [67]:
# testing accessing a representative point in Shapely
alamosq = hoods_by_city['San Francisco'][0][1].representative_point()
latitude = alamosq.x
longitude = alamosq.y

WALKSCORE_API_KEY = 'd6b2f4898db90acf861c91ce9e59e7dd'

def walkscore_request(address, latitude, longitude):
    response = requests.get('http://api.walkscore.com/score', params={
        'format': 'json',
        'address': address,
        'lat': latitude,
        'lon': longitude,
        'wsapikey': WALKSCORE_API_KEY,
        })
    return response.json()

# hood = hoods_by_city['San Francisco'][1][1]
# centroid = hood.centroid
# print centroid, centroid.x
walkscores = {}
for hood in hoods_by_city['San Francisco']:
    point = hood[1].centroid
    latitude = point.x
    longitude = point.y
    walkscore = walkscore_request("", latitude, longitude)['walkscore']
    print walkscore
    if hood[0] not in walkscores:
        walkscores[hood[0]] = walkscore

print walkscores
# It seems that you can pass an empty string for address and they'll figure it out themselves
#pprint(walkscore_request("747 Howard St, San Francisco, CA 94103", 37.783817, -122.4007327))

95
93
85
88
90
82
92
78
64
89
97
81
57
99
98
97
74
99
94
59
75
36
80
59
67
96
97
21
79
74
65
98
91
96
93
94
79
12
90
76
97
97
66
85
45
54
76
97
84
53
81
95
99
95
93
65
77
57
88
80
97
85
91
32
64
83
22
93
94
83
73
79
90
98
83
75
99
67
98
64
92
98
65
73
18
34
98
77
41
73
72
96
{'Ingleside Terrace': 65, 'Forest Knolls': 36, 'West Portal': 92, 'Golden Gate Heights': 59, 'Cow Hollow': 97, 'Cole Valley/Parnassus Heights': 91, 'Bernal Heights': 90, 'South of Market': 98, 'Duboce Triangle': 98, 'Eureka Valley / Dolores Heights': 97, 'Visitacion Valley': 64, 'Marina': 97, 'Merced Manor': 85, 'Outer Parkside': 57, 'Little Hollywood': 72, 'Saint Francis Wood': 83, 'Forest Hill': 59, 'Mission Bay': 76, 'Forest Hills Extension': 75, 'Miraloma Park': 54, 'Pine Lake Park': 32, 'Outer Sunset': 80, 'Lincoln Park': 18, 'Stonestown': 83, 'Outer Richmond': 88, 'Buena Vista Park/Ashbury Heights': 82, 'Bayview Heights': 73, 'Jordan Park / Laurel Heights': 94, 'South Beach': 90, 'Portola': 64, 'Sunnyside': 7

In [69]:
ws = pd.DataFrame(walkscores.items(), columns=['Neighborhood', 'Walk Score'])

In [70]:
ws.head()

,Neighborhood,Walk Score
0,Ingleside Terrace,65
1,Forest Knolls,36
2,West Portal,92
3,Golden Gate Heights,59
4,Cow Hollow,97


In [10]:
ws2 = pd.read_csv('../data/walk_scores_Seattle.csv')

In [11]:
ws2.head()

,Neighborhood,Walk Score
0,Eastlake,76
1,Harrison/Denny-Blaine,76
2,Minor,88
3,Gatewood,63
4,Fremont,78


In [21]:
WALKSCORE_API_KEY = 'd6b2f4898db90acf861c91ce9e59e7dd'

def transit_score_request(latitude, longitude, city, state):
    response = requests.get('http://transit.walkscore.com/transit/score/', params={
        'lat': latitude,
        'lon': longitude,
        'city': city,
        'state': state,
        'wsapikey': WALKSCORE_API_KEY,
    })
    return response.json()

pprint(transit_score_request(37.783817, -122.4007327, "San Francisco", "CA"))

{u'description': u"Rider's Paradise",
 u'help_link': u'https://www.redfin.com/how-walk-score-works',
 u'logo_url': u'https://cdn.walk.sc/images/transit-score-logo.png',
 u'summary': u'116 nearby routes: 99 bus, 14 rail, 3 other',
 u'transit_score': 100,
 u'ws_link': u'https://www.walkscore.com/score/loc/lat=37.7838/lng=-122.4007/?utm_source=wolak.net&utm_medium=ts_api&utm_campaign=ts_api'}
